## Fake News Detection Using NLP

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/FakeNewsDataset/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Drop Nan Values

In [5]:
df=df.dropna()

Get the Independent Features

In [6]:
X=df.drop('label',axis=1)

 Get the Dependent features

In [7]:
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

#Data Pre-processing

In [37]:
import tensorflow as tf

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
tf.__version__

'2.3.0'

Vocabulary size

In [13]:
voc_size=5000

###Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus[3]

'civilian kill singl us airstrik identifi'

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3664, 225, 3806, 1692, 2957, 4055, 1263, 2106, 2175, 1325],
 [1573, 4721, 1325, 4247, 238, 3208, 1605],
 [4360, 1987, 3687, 4789],
 [2361, 4934, 3824, 2063, 4015, 3727],
 [760, 238, 3248, 1869, 3906, 4371, 238, 4821, 4122, 384],
 [1233,
  3171,
  1301,
  291,
  1297,
  3176,
  3785,
  3533,
  1405,
  2282,
  209,
  1538,
  186,
  166,
  1605],
 [2416, 1173, 1622, 3652, 3569, 886, 1528, 3698, 4005, 4455, 3201],
 [2347, 866, 791, 3976, 3417, 4927, 3176, 4628, 4005, 4455, 3201],
 [2426, 2977, 4467, 4562, 3611, 4082, 3974, 740, 3176, 2666],
 [826, 544, 1730, 1431, 1404, 3794, 2682, 33],
 [2003, 259, 2701, 1179, 4623, 902, 1843, 2716, 1698, 3235, 730],
 [2063, 218, 2957, 4082, 3176, 3417],
 [1844, 23, 1750, 4885, 2124, 487, 1426, 614, 3262],
 [2067, 2170, 2435, 2500, 1680, 4546, 2685, 4005, 4455, 3201],
 [2206, 4453, 1582, 1189, 3336, 4005, 4455, 3201],
 [99, 2151, 4419, 3192, 456, 2415, 1686, 4391, 100, 1392],
 [1148, 2516, 4721],
 [922, 3344, 2253, 2269, 3176, 4592, 2973, 1605],
 [4541,

## Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2106 2175 1325]
 [   0    0    0 ...  238 3208 1605]
 [   0    0    0 ... 1987 3687 4789]
 ...
 [   0    0    0 ... 4005 4455 3201]
 [   0    0    0 ... 4985  884 3832]
 [   0    0    0 ... 4147 2873 3445]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3664,
        225, 3806, 1692, 2957, 4055, 1263, 2106, 2175, 1325], dtype=int32)

# Model

In [24]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [29]:
from tensorflow.keras.layers import Dropout

In [30]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Model Training

In [31]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
192/192 [==============================] - 6s 34ms/step - loss: 0.3407 - accuracy: 0.8380 - val_loss: 0.2018 - val_accuracy: 0.9173
Epoch 2/50
192/192 [==============================] - 6s 31ms/step - loss: 0.1501 - accuracy: 0.9397 - val_loss: 0.1889 - val_accuracy: 0.9233
Epoch 3/50
192/192 [==============================] - 6s 32ms/step - loss: 0.1121 - accuracy: 0.9568 - val_loss: 0.2093 - val_accuracy: 0.9152
Epoch 4/50
192/192 [==============================] - 6s 31ms/step - loss: 0.0838 - accuracy: 0.9682 - val_loss: 0.2487 - val_accuracy: 0.9150
Epoch 5/50
192/192 [==============================] - 6s 32ms/step - loss: 0.0646 - accuracy: 0.9778 - val_loss: 0.2418 - val_accuracy: 0.9120
Epoch 6/50
192/192 [==============================] - 6s 32ms/step - loss: 0.0474 - accuracy: 0.9840 - val_loss: 0.3334 - val_accuracy: 0.9107
Epoch 7/50
192/192 [==============================] - 6s 32ms/step - loss: 0.0359 - accuracy: 0.9878 - val_loss: 0.3795 - val_accuracy: 0.9109

### Performance Metrics And Accuracy

In [32]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [33]:
y_pred[:10]

array([[1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int32)

In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_test,y_pred)

array([[3103,  316],
       [ 274, 2342]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9022369511184756